In [1]:
# !pip install gradio

In [31]:
import torch
import gradio as gr
from diffusers import StableDiffusionPipeline, UNet2DConditionModel
from peft import get_peft_model, LoraConfig
from safetensors.torch import load_file
import uuid


In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
#Paths
base_model = "runwayml/stable-diffusion-v1-5"
lora_path ="/content/drive/MyDrive/lora-anime"

In [19]:
# === Load base UNet ===
unet = UNet2DConditionModel.from_pretrained(
    base_model,
    subfolder="unet",
    torch_dtype=torch.float16
)

In [20]:
# === Apply LoRA wrapper ===
lora_config = LoraConfig(
    r=4,
    lora_alpha=16,
    target_modules=["to_q", "to_k", "to_v"],
    lora_dropout=0.1,
    bias="none"
)
unet = get_peft_model(unet, lora_config)

# Re-Saving the trained weights

In [14]:
# #Load weights
# unet.load_adapter("/content/drive/MyDrive/lora-anime", adapter_name="default", is_trainable=False)

In [15]:
# unet.save_lora_adapter("/content/drive/MyDrive/lora-anime")

In [16]:
# # === Load adapter weights ===
# unet.load_state_dict(torch.load(f"{lora_path}/pytorch_model.bin"), strict=False)
# unet = unet.merge_and_unload()  # Merge LoRA into UNet

#Gradio App

In [22]:
#Load the LoRA Adapter weights
lora_weights = load_file(f"{lora_path}/pytorch_lora_weights.safetensors")
unet.load_state_dict(lora_weights, strict=False)

_IncompatibleKeys(missing_keys=['base_model.model.conv_in.weight', 'base_model.model.conv_in.bias', 'base_model.model.time_embedding.linear_1.weight', 'base_model.model.time_embedding.linear_1.bias', 'base_model.model.time_embedding.linear_2.weight', 'base_model.model.time_embedding.linear_2.bias', 'base_model.model.down_blocks.0.attentions.0.norm.weight', 'base_model.model.down_blocks.0.attentions.0.norm.bias', 'base_model.model.down_blocks.0.attentions.0.proj_in.weight', 'base_model.model.down_blocks.0.attentions.0.proj_in.bias', 'base_model.model.down_blocks.0.attentions.0.transformer_blocks.0.norm1.weight', 'base_model.model.down_blocks.0.attentions.0.transformer_blocks.0.norm1.bias', 'base_model.model.down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_q.base_layer.weight', 'base_model.model.down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_q.lora_A.default.weight', 'base_model.model.down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_q.lora_B.default.weight', 'ba

In [23]:
#Merge LoRA and unwrap peft
unet = unet.merge_and_unload()

In [24]:
#Load Full pipeline with Merged UNet
pipe = StableDiffusionPipeline.from_pretrained(
    base_model,
    unet=unet,
    torch_dtype=torch.float16
).to("cuda")

pipe.enable_attention_slicing()

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [34]:
#Inference Dunction
def generate(prompt, negative_prompt, guidance_scale, steps, seed):
  generator = torch.manual_Seed(seed) if seed != -1 else None
  output = pipe(
      prompt=prompt,
      negative_prompt=negative_prompt,
      guidance_scale = guidance_scale,
      num_inference_steps=steps,
      generator=generator
  )
  return output.images[0]
  # image = output.images[0]

  # #Save image to temporary file
  # filename = f"{uuid.uuid4().hex}.png"
  # filepath = os.path.join("outputs", filename)
  # os.makedirs("outputs", exists_ok=True)
  # image.save(filepath)

  # return image, filepath


In [35]:
#Gradio Interface
gr.Interface(
    fn=generate,
    inputs=[
        gr.Textbox(label="Prompt", placeholder="A cute anime girl with pink hair, black eyes"),
        gr.Textbox(label="Negative Prompt", placeholder="blurry, bad anatomy, low res"),
        gr.Slider(1.0, 20.0, value=7.5, step=0.5, label="Guidance Scale"),
        gr.Slider(10, 50, value=30, step=1, label="Inference Steps"),
        gr.Slider(-1, 999999, step=1, value=-1, label="Seed (-1 for random)")
    ],
    outputs = [
        gr.Image(label="Generated Image")
        #gr.File(label="Download Image")
    ],
    title="🎨 Anime Face Genrator with LoRA",
    theme = "default"
).launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ac598d113f026217a1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
